In [1]:
import cv2
import os
import fnmatch
import re
import trackpy as tp 
import numpy 
import matplotlib.pylab as plt 
import pims_nd2
import numpy as np
from matplotlib.widgets import TextBox
import pandas as pd

In [2]:
# File folder

folder = r'C:\\Users\\Julia\\Downloads\\'

# Filename

filename = r'kirill_002'


# Fluorescence channel number

channel = 0

In [10]:
%matplotlib qt

class ImageProcessingGUI():
    def __init__(self, img, img_labels, trajectories, dictionary_frames):

        self.fig = plt.figure(figsize=(20,15))
        self.img = img
        self.img_labels = img_labels
        grid_sz = 15
        self.currentvideoindice = 1
        self.dictionary_frames = dictionary_frames

        self.trajectories = trajectories

        plt.subplot2grid((grid_sz, grid_sz), (0, 0), colspan=grid_sz, rowspan=grid_sz - 1)

        tp.plot_traj(self.trajectories, label=True)
        self.zerozero = plt.imshow(self.img[self.currentvideoindice], alpha=0.9)
        # self.imgprev = plt.imshow(self.img_labels[self.currentvideoindice], alpha = 0.6, cmap = 'jet')
        # plt.plot(trajectories)

        z_stack_ax = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 5 // 6))
        self.selected_cell_labels = []
        self.cell_labels = img_labels
        self.cell_mask = None

        self.CellBox = TextBox(z_stack_ax, 'Zstack number')
        self.CellBox.on_submit(self.on_z_stack_number_update_process)
        self.zerozero.axes.get_xaxis().set_visible(False)
        self.zerozero.axes.get_yaxis().set_visible(False)
        plt.title('Use sliders to choose current slide')
        plt.axis('off')
        # self.cid = self.fig.canvas.mpl_connect('button_press_event', self.on_mouse_click)
        slider_ax1 = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 1 // 6))
        button_ax = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 3 // 6))

        self.done_button = plt.Button(button_ax, 'Done')
        self.done_button.on_clicked(self.on_button_press)
        self.cutting = plt.Slider(slider_ax1, 'Current slide', 0, (np.shape(img_labels)[0] - 1),
                                  valinit=self.currentvideoindice, valstep=1)
        self.cutting.on_changed(self.on_slider_process)

        plt.show()
        self.fig.canvas.start_event_loop(timeout=-1)

    def on_button_press(self, event):
        plt.close()
        self.fig.canvas.stop_event_loop()

    def on_slider_process(self, val):
        self.currentvideoindice = int(val)
        self.zerozero.set_data(self.img[self.currentvideoindice])
        # self.imgprev.set_data(self.img_labels[self.currentvideoindice])
        plt.draw()

    def on_z_stack_number_update_process(self, val):
        try:
            self.label = int(eval(val))
            self.selected_cell_labels.append(self.label)
        except:
            print('Error1!')


def get_z_size(trace):
    frames = pims_nd2.ND2_Reader(trace)
    z = frames.sizes.get('m')

    pixel_size = frames[0].metadata['mpp']
    frames.close()
    return z, pixel_size


def getNd2efunc(z_stack, trace, channel_number=1):
    
    frames = pims_nd2.ND2_Reader(trace)
    x = frames.sizes.get('x')
    y = frames.sizes.get('y')

    frames.default_coords['m'] = z_stack
    frames.default_coords['c'] = 1
    t = frames.sizes.get('t')
    frames_psel = numpy.zeros((t, x, y))
    for i in range(t):
        pass
        frames_psel[i, :, :] = frames[i]
    frames.default_coords['c'] = channel_number    
    img = numpy.zeros((t, x, y))
    times = numpy.zeros((t))
    for i in range(t):
        pass
        img[i, :, :] = frames[i]
        times[i] = frames[i].metadata['t_ms'] / 1000
    # pixel size

    frames_fon = frames[:]
    frames.close()
    frames = img[:]

    frames_fon = frames.copy()
    f = tp.batch(frames_fon[:], 91)
    t = tp.link_df(f, 90, memory=4)
    f = tp.filter_stubs(t, 3)
    """
        img = img[:10]
        times = times[:10]"""
    dictionary = {}

    for i in (f['particle']):
        particle_center_list = []
        key = (f[f['particle'] == i])
        frame_list_for_particles = (f[f['particle'] == i])['frame']

        for j in frame_list_for_particles:
            centurx1 = key[key['frame'] == j]
            centurx = np.float64(centurx1['x'])
            century = np.float64(centurx1['y'])
            size = np.float64(centurx1['size'])
            particle_center_list.append([j, centurx, century, size])

        dictionary[i] = particle_center_list
    frame_labels = np.zeros_like(frames)

    binarize_video = np.zeros_like(frames_fon)

    for i in range(np.shape(binarize_video)[0]):
        ret, binarize_video[i] = cv2.threshold(np.uint8(frames_fon[i] * 255 / np.max(frames_fon[i])), 0, 255,
                                               cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binarize_video, frames_fon, frame_labels, f, dictionary, times, z_stack, frames_psel

In [12]:


TRACE = folder + '\\' + filename
z_max, mpp = get_z_size(TRACE + '.nd2')

all_z_stack_results = getNd2efunc(z_field_enum, TRACE + '.nd2', channel)


Frame 63: 4 trajectories present.


In [ ]:
if True:    
    all_elems = all_z_stack_results
    all_coords_array_t = []
    all_coords_array_x = []
    all_coords_array_y = []
    percent = []
    mean_intensity_thrombi = []
    binarize_video, frames_fon, frame_labels, f, dictionary, times, zstack_current_num, psel_vid = all_elems
    gui = ImageProcessingGUI(frames_fon, frame_labels, f, dictionary)
    label_list = gui.selected_cell_labels

    all_velocities_array = []
    for j in gui.selected_cell_labels:
        velocity_array = []
        x_array = []
        y_array = []
        t_array = []
        velocities = np.zeros(len(dictionary[j]))
        for frame_enum in range(1, len(dictionary[j])):
            frame_nr = dictionary[j][frame_enum][0]
            delta_t = times[frame_nr] - times[dictionary[j][frame_enum - 1][0]]
            delta_x = dictionary[j][frame_enum][1] - dictionary[j][frame_enum - 1][1]
            delta_y = dictionary[j][frame_enum][2] - dictionary[j][frame_enum - 1][2]

            velocity = np.sqrt(delta_x ** 2 + delta_y ** 2)
            velocity = velocity * mpp / delta_t
            velocity_array.append(velocity)
            t_array.append(times[frame_nr])
            x_array.append(dictionary[j][frame_enum][1])
            y_array.append(dictionary[j][frame_enum][2])
            mask = np.zeros_like(binarize_video[frame_nr])
            size_leuco = dictionary[j][frame_enum][3]
            x_this_slide = dictionary[j][frame_enum][1]
            y_this_slide = dictionary[j][frame_enum][2]
            mask[int(y_this_slide - size_leuco * 1.2):int(y_this_slide + size_leuco * 1.2),
            int(x_this_slide - size_leuco * 1.2):int(x_this_slide + size_leuco * 1.2)] += 1
            binarize_video[frame_nr][mask == 1] *= 0

        all_velocities_array.append(velocity_array)
        all_coords_array_t.append(t_array)
        all_coords_array_x.append(x_array)
        all_coords_array_y.append(y_array)

    for j in range(np.shape(binarize_video)[0]):
        frame_thrombus_per_cent = len(binarize_video[j][binarize_video[j] != 0]) / (
                    np.shape(binarize_video)[2] * np.shape(binarize_video)[1])
        psel_vid[j][binarize_video[j]==0]*=0
        frame_mean_intensity = np.sum(psel_vid[j])/len(binarize_video[j][binarize_video[j] != 0])
        mean_intensity_thrombi.append(frame_mean_intensity)
        percent.append(frame_thrombus_per_cent)


  